In [1]:
import pandas as pd
import os

env = os.environ.get('Environment')
print(f"Environment: {env!r}")

Environment: 'prod'


In [2]:
%reload_ext jupyter_dmdg

## Non-compliant equipment

In [3]:
# non-compliant equipment at L3

In [4]:
%%athena_to_df --out L3
WITH level_3_trigrams AS (
    SELECT 
        trigram 
    FROM "prod-plantco-referential-data"."prod_plantco_referential_hierarchy"
    WHERE level = '3'
)
SELECT DISTINCT
    site_id,
    location_hierarchy, 
    SUBSTRING(premise_id,1,3) AS premise 
FROM "prod_plant_connectivity_prepared_data"."process_events" events 
LEFT OUTER JOIN level_3_trigrams t ON t.trigram = SUBSTRING(events.premise_id,1,3)
WHERE 
    t.trigram IS NULL
    AND premise_id != ''
    AND secondary_function_id IS NULL

In [5]:
# non-compliant equipment at L4

In [6]:
%%athena_to_df --out L4
WITH level_4_trigrams AS (
    SELECT 
        trigram 
    FROM "prod-plantco-referential-data"."prod_plantco_referential_hierarchy"
    WHERE level = '4'
)
SELECT DISTINCT
    site_id,
    location_hierarchy, 
    SUBSTRING(main_function_id,1,3) AS main_function
FROM "prod_plant_connectivity_prepared_data"."process_events" events 
LEFT OUTER JOIN level_4_trigrams t ON t.trigram = SUBSTRING(events.main_function_id,1,3)
WHERE 
    t.trigram IS NULL
    AND main_function_id != ''
    AND secondary_function_id IS NULL

In [7]:
# non-compliant equipment

In [8]:
non_compliant_equipment = pd.merge(L3, L4, on=['site_id', 'location_hierarchy'], how='outer')

## Statistics for equipment

In [9]:
# count how many distinct location_hierarchy there are per site_id in the process events table

In [10]:
%%athena_to_df --out all_LH
SELECT 
    site_id, 
    COUNT(DISTINCT location_hierarchy) AS nb_equipment
FROM "prod_plant_connectivity_prepared_data"."process_events"
WHERE 
    premise_id != ''
    AND secondary_function_id IS NULL
GROUP BY 1

In [11]:
# count how many non-compliant equipment per site_id there are

In [12]:
count_non_compliant_equipment = non_compliant_equipment.groupby('site_id').size().reset_index(name='nb_non_compliant_equipment')

In [13]:
# calculate the compliance percentage

In [14]:
for_statistics = all_LH.merge(count_non_compliant_equipment, on='site_id', how='outer')
for_statistics['nb_non_compliant_equipment'].fillna(0, inplace=True)
for_statistics['nb_non_compliant_equipment'] = for_statistics['nb_non_compliant_equipment'].astype(int)
for_statistics['pct_compliance'] = 100-(100*for_statistics['nb_non_compliant_equipment']/for_statistics['nb_equipment'])
for_statistics['pct_compliance'] = for_statistics['pct_compliance'].astype(float).round(2)

## Have both list with non-compliant equipment and compliance statistics for equipment in the dedicated workspace

In [15]:
from src.utils import write_into_table
workspace_prefix = 'public-plant-co-data-quality-20230520193210554'

In [16]:
# list with non-compliant equipment in the dedicated workspace

In [17]:
equipment_table_name = 'non_compliant_equipment'
write_into_table(
    non_compliant_equipment,
    equipment_table_name.lower(),
    workspace_prefix
)

In [18]:
# compliance statistics for equipment in the dedicated workspace

In [19]:
equipment_stats_table_name = 'stats4equipment'
write_into_table(
    for_statistics,
    equipment_stats_table_name.lower(),
    workspace_prefix
)